In [2]:
import os
import tvm
from tvm.script import ir as I
from tvm.script import tir as T
from tvm import autotvm, auto_scheduler
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import meta_schedule as ms
from tvm.ir import IRModule
from tvm import relax
from tvm import te
from tvm import rpc
from tvm.contrib import utils

import numpy as np


In [15]:
def run_opencl():
    n = tvm.runtime.convert(1024)
    A = te.placeholder((n,), name="A")
    B = te.compute((n,), lambda i: A[i] + 1.0, name="B")
    s = te.create_schedule(B.op)
    # 注意：这是 rk3399 板的设置。你需要根据你的环境进行修改
    opencl_device_host = "10.158.176.30"
    opencl_device_port = 5001
    target = tvm.target.Target("opencl", host="llvm -mtriple=aarch64-linux-gnu")

    # 为上面的计算声明 "add one" 创建 schedule
    s = te.create_schedule(B.op)
    xo, xi = s[B].split(B.op.axis[0], factor=32)
    s[B].bind(xo, te.thread_axis("blockIdx.x"))
    s[B].bind(xi, te.thread_axis("threadIdx.x"))
    func = tvm.build(s, [A, B], target=target)

    remote = rpc.connect(opencl_device_host, opencl_device_port, key="")

    # 导出并上传
    temp = utils.tempdir()
    path = temp.relpath("lib_cl.tar")
    func.export_library(path)
    remote.upload(path)
    func = remote.load_module("lib_cl.tar")

    # 运行
    dev = remote.cl()
    a = tvm.nd.array(np.random.uniform(size=1024).astype(A.dtype), dev)
    b = tvm.nd.array(np.zeros(1024, dtype=A.dtype), dev)
    func(a, b)
    np.testing.assert_equal(b.numpy(), a.numpy() + 1)
    print("OpenCL test passed!")
run_opencl()

TVMError: Traceback (most recent call last):
  6: 0xffffffffffffffff
  5: 0x00000000005b7412
  4: __libc_start_main
  3: ffi_call
  2: operator()
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/runtime/rpc/rpc_socket_impl.cc:132
  1: tvm::runtime::RPCClientConnect(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, int, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, bool, tvm::runtime::TVMArgs)
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/runtime/rpc/rpc_socket_impl.cc:112
  0: tvm::runtime::RPCConnect(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, int, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, bool, tvm::runtime::TVMArgs)
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/runtime/rpc/rpc_socket_impl.cc:85
  File "/data/workspace/llm/github/new_wksp/tvm-unity/src/runtime/rpc/rpc_socket_impl.cc", line 85
TVMError: 